In [1]:
from python_files.data import data
data_ = data.copy()
from python_files.matrix.mat_split_data import X_train_mat, X_test_mat, y_train_mat, y_test_mat
from python_files.matrix.mat_col_list import mat_x_col_list, mat_col_list_norm, mat_col_list_std

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, QuantileTransformer, MaxAbsScaler, RobustScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from scikeras.wrappers import KerasRegressor

### Preprocessing

In [5]:
preprocessor_save_dist = ColumnTransformer(
    transformers=[
        ('normalization', PowerTransformer(), mat_col_list_norm), 
        ('scaling', StandardScaler(), mat_col_list_std) 
    ])

In [6]:
preprocessor_rob = ColumnTransformer(
    transformers=[
        ('scaling', RobustScaler(), mat_x_col_list)
    ])

In [7]:
preprocessor_maxabs = Pipeline(steps = [('scaling_1', StandardScaler(with_mean=True, with_std=False)),
                                     ('scaling_2', MaxAbsScaler())
                                    ])

In [8]:
preprocessor_std = ColumnTransformer(
    transformers=[
        ('scaling', StandardScaler(), mat_x_col_list)
    ])

### Model - MLP

In [81]:
def MLP_model():
    MLP_model = Sequential()
    
    MLP_model.add(Dense(8, input_shape = (12,), activation='selu', bias_initializer='he_uniform', kernel_initializer='he_normal'))
    MLP_model.add(Dropout(0.2))
   
    MLP_model.add(Dense(8, activation = 'tanh', bias_initializer='he_uniform', kernel_initializer='he_normal')) 
    MLP_model.add(Dropout(0.1))
    
    MLP_model.add(Dense(8, activation = 'relu', bias_initializer='he_uniform', kernel_initializer='he_normal')) 
    #MLP_model.add(Dropout(0.0))
    
    MLP_model.add(Dense(1, activation = 'linear', bias_initializer='he_uniform', kernel_initializer='he_uniform'))
    
    #adam = Adam(learning_rate = 0.01)
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    MLP_model.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics=['mean_absolute_error'])
    return MLP_model

In [82]:
model = TransformedTargetRegressor(regressor = KerasRegressor(model = MLP_model(), epochs=300, verbose=True),
                                  transformer = QuantileTransformer(n_quantiles = 716, output_distribution = "normal", random_state = 7))

In [83]:
Filling_Matrix_Regressor = Pipeline(steps=[('preprocessor', preprocessor_save_dist),
                           ('model', model)],
                           verbose=True)

### Training

In [84]:
Filling_Matrix_Regressor.fit(X_train_mat, y_train_mat)     #, model__epochs = 200, model__verbose = True) 

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s


INFO:tensorflow:Assets written to: C:\Users\WINDOW~1.1\AppData\Local\Temp\tmpdd_o8dcz\assets


INFO:tensorflow:Assets written to: C:\Users\WINDOW~1.1\AppData\Local\Temp\tmpdd_o8dcz\assets


Epoch 1/300
23/23 [==============================] - 1s 3ms/step - loss: 2.2646 - mean_absolute_error: 2.2646
Epoch 2/300
23/23 [==============================] - 0s 3ms/step - loss: 1.9266 - mean_absolute_error: 1.9266
Epoch 3/300
23/23 [==============================] - 0s 3ms/step - loss: 1.6671 - mean_absolute_error: 1.6671
Epoch 4/300
23/23 [==============================] - 0s 2ms/step - loss: 1.4360 - mean_absolute_error: 1.4360
Epoch 5/300
23/23 [==============================] - 0s 3ms/step - loss: 1.2931 - mean_absolute_error: 1.2931
Epoch 6/300
23/23 [==============================] - 0s 3ms/step - loss: 1.1619 - mean_absolute_error: 1.1619
Epoch 7/300
23/23 [==============================] - 0s 3ms/step - loss: 1.0811 - mean_absolute_error: 1.0811
Epoch 8/300
23/23 [==============================] - 0s 3ms/step - loss: 0.9791 - mean_absolute_error: 0.9791
Epoch 9/300
23/23 [==============================] - 0s 3ms/step - loss: 0.9541 - mean_absolute_error: 0.9541
Epoch 10/3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('normalization',
                                                  PowerTransformer(),
                                                  Index(['Плотность, кг/м3', 'модуль упругости, ГПа',
       'Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2',
       'Температура вспышки, С_2', 'Модуль упругости при растяжении, ГПа',
       'Прочность при растяжении, МПа', 'Потребление смолы, г/м2',
       'Шаг нашивки', 'Плотность нашивки...
                                                 ('scaling', StandardScaler(),
                                                  Index(['Поверхностная плотность, г/м2', 'Угол нашивки, град'], dtype='object'))])),
                ('model',
                 TransformedTargetRegressor(regressor=KerasRegressor(epochs=300, model=<keras.engine.sequential.Sequential object at 0x0000005B5DAF5F10>, verbose=True),
                                            transformer=QuantileTransformer(n_quantiles=716,
                                                                            output_distribution='normal',
                                                                            random_state=7)))],
         verbose=True)

### Prediction

In [85]:
preds = Filling_Matrix_Regressor.predict(X_test_mat)

10/10 [==============================] - 0s 2ms/step


In [86]:
preds_train = Filling_Matrix_Regressor.predict(X_train_mat)

23/23 [==============================] - 0s 2ms/step


In [87]:
print(y_test_mat.iloc[100])

1.27327251867668


In [88]:
Filling_Matrix_Regressor.predict(X_test_mat.iloc[100 : 100 + 1])

1/1 [==============================] - 0s 26ms/step


array([2.8020458], dtype=float32)

### Evaluation

In [89]:
r2_score(y_train_mat, preds_train), r2_score(y_test_mat, preds)

(0.11899889324533042, -0.11681830369144053)

In [90]:
mean_absolute_error(y_train_mat, preds_train), mean_absolute_error(y_test_mat, preds)

(0.6830669942374984, 0.7326531817461217)

In [91]:
mean_squared_error(y_train_mat, preds_train), mean_squared_error(y_test_mat, preds)

(0.764053401480636, 0.8416047671970848)

In [92]:
mean_absolute_error(y_test_mat, preds) * 100 / (y_test_mat.max() - y_test_mat.min())

14.083149572890825